## Methods to to solve for Klinkenberg Permeability from Permeability to air:

Recently, while exploring PermInc.com, I stumbled upon an intriguing approach utilizing the Newton-Raphson method to convert permeability to air measurements into Klinkenberg permeability. While the website provided an outline, understanding and implementing the concept required translating it into Python code. Additionally, delving deeper, I reviewed API Recommended Practices RP-40.

In RP-40's appendix, Jones introduced a correlation for helium's b (Klinkenberg gas slippage factor) using an equation (A-7):

    b_Helium = 16.4 * Kl^(-0.382)

Where 'b' represents the Klinkenberg gas slippage factor in psi and 'Kl' denotes the Klinkenberg Permeability in mD. The document suggests calculating 'b_air' by substituting 16.4 with 5.71:

    b_air = 5.71 * Kl^(-0.382)

Applying equation 21 of RP-40, which relates to air:

    Kair = Kl * (1 + b_air / p_mean)

For air, this equation simplifies to:

    Kair = Kl + 5.71 * Kl^(1 - 0.382) / p_mean

This equation can be reformulated as:

    5.71 * Kl^0.618 + Kl * p_mean - Kair * p_mean = 0

The Newton-Raphson method necessitates the derivative of the function concerning the variable being solved for (in this instance, Kl) to iteratively converge towards the solution.

The derivative of the function f(Kl) = 5.71 * Kl^0.618 + Kl * p_mean - Kair * p_mean with respect to Kl is derived as follows:

    f'(Kl) = df(Kl)/dKl = 5.71 * 0.618 * Kl^(-0.382) + p_mean

This derivative, f'(Kl), is crucial in updating the guessed value for Kl iteratively. It assists in determining how to update the guess to converge towards the root (or solution) of the equation.

The python code for the Newton-Raphson, SciPy Optimization  and excel spreadsheet can be found on this GitHub repository.

All 3 coded examples provide the same estimated for Klinkenberg permeability based on the given data from RP-40 utilized in equation 31, resulting in an estimated Kl of 10.61, where in RP-40 the Klinkenberg Permeability is 10.62, indicating a close approximation.

For further details, refer to [PermInc.com](https://perminc.com/resources/fundamentals-of-fluid-flow-in-porous-media/chapter-2-the-porous-medium/permeability/) and the work of Jones (SPE 16949, 1987).

```python
    # Given data
    K_air_mD   = 11.97  # Measured permeability to air in mD
    p_mean_psi = 18     # Mean pressure in psi

    # Function to solve for Kl using Newton-Raphson method
    def calculate_Klinkenberg(K_air_mD, p_mean_psi):
        # Initial guess for Kl
        Kl = K_air_mD / 2  # Initial guess can be adjusted if needed

        # Define the function that needs to be solved to find Kl
        def f(Kl_guess):
            return 5.71 * (Kl_guess**(0.618)) + Kl_guess * p_mean_psi - K_air_mD * p_mean_psi

        # Define the derivative of the function for Newton-Raphson method
        def df(Kl_guess):
            return 5.71 * 0.618 * (Kl_guess**(-0.382)) + p_mean_psi

        # Set tolerance and maximum iterations for convergence
        tolerance = 1e-6
        max_iterations = 100
        
        iteration = 0
        while True:
            Kl_new = Kl - f(Kl) / df(Kl)

            if abs(Kl_new - Kl) < tolerance or iteration >= max_iterations:
                break

            Kl = Kl_new
            iteration += 1

        return Kl_new

    # Calculate Klinkenberg permeability using the function
    estimated_Kl = calculate_Klinkenberg(K_air_mD, p_mean_psi)
    print(f"The estimated Klinkenberg permeability is {estimated_Kl:.2f} mD.")
```

This code defines a function `calculate_Klinkenberg` that uses the Newton-Raphson method to iteratively solve for Kl given the provided equations and parameters. The initial guess for Kl is set to half the measured permeability to air, but this can be adjusted depending on the specific characteristics of the problem.

Running this code will output the estimated Klinkenberg permeability in milliDarcy (mD) based on the provided equations and iterative method. 

In the Newton-Raphson method, the derivative of the function with respect to the variable being solved for (in this case, Kl) is required to iteratively converge towards the solution. 

In the given equation:

    f(Kl) = 5.71 * Kl**{0.618} + Kl * p_mean - K_air * p_mean

The derivative of this function with respect to Kl can be calculated using standard differentiation rules. The derivative of the function f(Kl) with respect to Kl is denoted as f'(Kl) or df(Kl)/dKl. Applying the power rule and the derivative of a constant multiplied by a function, we get:

    f'(Kl) = df(Kl)/dKl = 5.71 * 0.618 * Kl**{0.618 - 1} + p_mean
    
    f'(Kl) = df(Kl)/dKl = 5.71 * 0.618 * Kl**{-0.382} + p_mean

This derivative, f'(Kl), is used in the Newton-Raphson method to update the guess for Kl iteratively. The derivative helps to find the rate of change of the function with respect to Kl at a particular point, which aids in determining how to update the guess to converge towards the root (or solution) of the equation.

In the provided Python code, the derivative function is defined as `df(Kl_guess)` and is used within the Newton-Raphson iteration to calculate the updated value of Kl. This iterative process continues until the difference between successive approximations falls below a specified tolerance or until a maximum number of iterations is reached.

In [1]:
# Given data
K_air_mD = 11.97  # Measured permeability to air in mD
p_mean_psi = 18.0  # Mean pressure in psi


# Function to solve for Kl using Newton-Raphson method
def calculate_Klinkenberg(K_air_mD, p_mean_psi):
    # Initial guess for Kl
    Kl = K_air_mD / 2  # Initial guess can be adjusted if needed

    # Define the function that needs to be solved to find Kl using 5.71 for the b multiplier for b_air
    def f(Kl_guess):
        return 5.71 * (Kl_guess ** 0.618) + Kl_guess * p_mean_psi - K_air_mD * p_mean_psi

    # Define the derivative of the function for Newton-Raphson method
    def df(Kl_guess):
        return 5.71 * 0.618 * (Kl_guess ** -0.382) + p_mean_psi

    # Set tolerance and maximum iterations for convergence
    tolerance = 1e-6
    max_iterations = 100
    iteration = 0
    while True:
        Kl_new = Kl - f(Kl) / df(Kl)

        if abs(Kl_new - Kl) < tolerance or iteration >= max_iterations:
            break

        Kl = Kl_new
        iteration += 1
        
        print(iteration)
        
    return Kl_new

# Calculate Klinkenberg permeability using the function
estimated_Kl = calculate_Klinkenberg(K_air_mD, p_mean_psi)
print(f"The estimated Klinkenberg permeability is {estimated_Kl:.2f} mD.")


1
2
3
The estimated Klinkenberg permeability is 10.61 mD.


## Use of SciPy Optimization:

In the context of solving the same equation for root-finding or minimizing the function representing the equation, it's expected that different optimization techniques or methods that converge successfully should provide similar or identical solutions if the methods are implemented correctly and the functions are well-behaved.

In the next examples we are using a different optimization approaches (`root_scalar` with Newton-Raphson, and `minimize` with least squares minimization), the goal was to find the root of the equation f(x) = 0 by employing distinct optimization strategies. If these methods converge successfully, they should yield very close or identical estimates for the root, given the same initial conditions and convergence criteria.

Therefore, observing similar results between the different optimization methods applied to the same problem (root-finding) confirms that the methods are effectively finding the root or solution to the equation f(x) = 0 in a consistent manner.

This consistency in results across different optimization techniques provides additional confidence in the accuracy of the estimated solution for the root of the equation f(x) = 0 given the specific function and initial conditions used in the examples.

In [2]:
from scipy.optimize import root_scalar

# Define the function and its derivative
def f(x):
    return 5.71 * (x ** 0.618) + x * p_mean_psi - K_air_mD * p_mean_psi

def df(x):
    return 5.71 * 0.618 * (x ** -0.382) + p_mean_psi

# Given data
K_air_mD = 11.97  # Measured permeability to air in mD
p_mean_psi = 18.0  # Mean pressure in psi

# Initial guess for the root
x_initial_guess = K_air_mD / 2  # Initial guess can be adjusted if needed

# Use scipy.optimize.root_scalar to find the root
root = root_scalar(f, fprime=df, x0=x_initial_guess, method='newton')

if root.converged:
    estimated_Klinkenberg = root.root
    print(f"The estimated Klinkenberg permeability is {estimated_Klinkenberg:.2f} mD.")
else:
    print("Root finding did not converge.")


The estimated Klinkenberg permeability is 10.61 mD.


In [3]:
from scipy.optimize import minimize

# Define the function
def objective_function(x):
    return (5.71 * (x ** 0.618) + x * p_mean_psi - K_air_mD * p_mean_psi) ** 2
    #return 5.71 * (x ** 0.618) + x * p_mean_psi - K_air_mD * p_mean_psi

# Given data
K_air_mD = 11.97  # Measured permeability to air in mD
p_mean_psi = 18.0  # Mean pressure in psi

# Initial guess for the root
x_initial_guess = K_air_mD / 2  # Initial guess can be adjusted if needed

# Use scipy.optimize.minimize to find the root (solution to f(x) = 0)
result = minimize(objective_function, x_initial_guess, method='Nelder-Mead', tol=1e-6)

if result.success:
    estimated_Klinkenberg = result.x[0]
    print(f"The estimated Klinkenberg permeability is {estimated_Klinkenberg:.2f} mD.")
else:
    print("Optimization did not converge.")


The estimated Klinkenberg permeability is 10.61 mD.


The choice of optimization method depends on various factors such as the properties of the function, availability of derivatives, computational resources, convergence behavior, and the specific problem at hand. Each optimization method has its strengths and limitations. Here's a summary of considerations for the methods discussed:

1. **Newton-Raphson Method:**
   - Requires both the function and its derivative.
   - Converges rapidly for well-behaved functions when a good initial guess is provided.
   - May not converge or converge to a local minimum/maximum for poorly behaved functions.
   - Efficient if derivatives can be computed accurately.

2. **`scipy.optimize.root_scalar`:**
   - Specifically designed for root-finding.
   - Utilizes different algorithms including Newton-Raphson, secant, Brent's method, etc.
   - Requires fewer function evaluations than `scipy.optimize.minimize`.
   - May be more straightforward to use for root-finding problems.

3. **`scipy.optimize.minimize`:**
   - General-purpose optimization tool that can solve root-finding problems by minimizing a function (like least squares minimization).
   - Doesn't require derivatives in some methods (e.g., Nelder-Mead) but can benefit from derivatives for faster convergence in others.
   - Suitable for more general optimization problems beyond root-finding.

Given your specific problem of finding the root of the equation \(f(x) = 0\) for the Klinkenberg permeability, if you have access to derivatives and are mainly focused on root-finding, using `scipy.optimize.root_scalar` with the Newton-Raphson method could be a good choice. This method tends to converge quickly if a good initial guess is provided and if the function behaves well around the root.

However, for a more general optimization problem or when derivatives are not easily computable, using `scipy.optimize.minimize` with appropriate methods, such as Nelder-Mead, can be versatile and may not require derivative information.

Ultimately, the choice should be based on the specific characteristics of your function, computational requirements, availability of derivative information, and the level of confidence you seek in the convergence behavior of the optimization method. It might be beneficial to experiment with different methods and compare their performance for your particular problem to determine the most suitable one.